# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 사용할 변수 선택하기
- 모델링
- 추론
- 결과 저장하기
- 결과 그 이후

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
# basic
import os, random
import pandas as pd
import numpy as np
import torch

# model
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

# eval metric
from sklearn.metrics import confusion_matrix

# k-fold by timeseries split
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold

# graph
import shap
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [3]:
# load makes data
train = pd.read_csv('../data/train_preprocessed_data.csv')
test = pd.read_csv('../data/test_preprocessed_data.csv')

### 3. 사용할 변수 선택하기

데이터에서 필요한 칼럼을 선택하여 적용하기로 한다.

In [4]:
# onehotencoder in ground
# train['A'] = np.where(train['ground'] == 'A', 1, 0)
# train['B'] = np.where(train['ground'] == 'B', 1, 0)
# train['C'] = np.where(train['ground'] == 'C', 1, 0)
# train['D'] = np.where(train['ground'] == 'D', 1, 0)

# test['A'] = np.where(test['ground'] == 'A', 1, 0)
# test['B'] = np.where(test['ground'] == 'B', 1, 0)
# test['C'] = np.where(test['ground'] == 'C', 1, 0)
# test['D'] = np.where(test['ground'] == 'D', 1, 0)

#### 단순히 I, J년 train, K년 valid로 활용할 때 사용할 코드

In [5]:
# label 선택하기
use_label_x = ['hm', 're', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'ground', # 'A', 'B', 'C', 'D',
       'dew_point', 'sin_time', 'cos_time', 'sin_month', 'cos_month', 'diff_air-dew', 'diff_ts-dew', 'fog_risk',
       'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'
       ]

use_label_y = ['class']

In [6]:
# train_x, train_y, test_x 만들자
# train: I,J년도, valid: K년도, test: L년도
# train_x = train.loc[(train['year'].isin(['I', 'J'])), use_label_x]
# train_y = train.loc[(train['year'].isin(['I', 'J'])), use_label_y]

# valid_x = train.loc[train['year'].isin(['K']), use_label_x]
# valid_y = train.loc[train['year'].isin(['K']), use_label_y]

# test_x = test[use_label_x]

#### TimeSeriesSplit

In [7]:
# 지정 label
use_label = [
    'hm', 're', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'ground', # 'A', 'B', 'C', 'D',
    'dew_point', 'sin_time', 'cos_time', 'sin_month', 'cos_month', 'diff_air-dew', 'diff_ts-dew', 'fog_risk',
    # 'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog',
    'class'
]

cat_features = [
    're', 'ground'#, # 'A', 'B', 'C', 'D',
    # 'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'
]

In [8]:
# 사용할 변수만 넣어주기
use_train = train[use_label]
use_test = test[use_label]

In [9]:
# parameter 지정
# gap: train 이후 몇개를 사용하지 않을것인지 정하기 위한 파라미터
tscv = TimeSeriesSplit(n_splits = 6)

In [10]:
# train_x, train_y, test_x 만들자
train_x = use_train[use_train.columns.difference(['class'])]
train_y = use_train['class']

test_x = test[use_train.columns.difference(['class'])]

### 4. 모델링
Catboost는 시계열 데이터 또는 범주형 데이터에 좋은 성능을 가져오는 모델로 알려져 있기 때문에 적용하도록 한다.

timeseries split을 활용해 적합을 진행해보도록 한다.

In [11]:
# custom metric
class CSIMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        best_class = np.zeros(len(approxes[0]))
        
        for i in range(len(approxes[0])):
            approx_i = [approxes[j][i] for j in range(len(approxes))]
            best_class[i] = np.argmax(np.array(approx_i))
        
        accuracy_sum = 0
        weight_sum = 0 

        for i in range(len(target)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            if best_class[i] != '4' or best_class[i] != 4:
                accuracy_sum += w * (best_class[i] == target[i])

        return accuracy_sum, weight_sum

In [14]:
# time series cv fitting models

# 해당 스코어와 모델 저장 리스트
scores = []
models = []

# split마다 모델 적합하기
for train_idx, valid_idx in tscv.split(use_train):
    print("="*50)
    
    # 가중치 만들기
    class_weights = [100, 65, 64, 0.25]


    # best parameter - 웬만하면 이 파라미터로 계속 진행할 예정
    cbrm_trial_params = {'objective': 'MultiClass',
                        'depth': 12, 
                        'boosting_type': 'Plain', 
                        'bootstrap_type': 'Bernoulli', 
                        'iterations': 2178, 
                        'learning_rate': 0.03978756063905378, 
                        'reg_lambda': 95.06719663989301, 
                        'subsample': 0.980507422817941, 
                        'random_strength': 42.091243716971206, 
                        'min_data_in_leaf': 61, 
                        'leaf_estimation_iterations': 1,
                        'cat_features':cat_features,
                        'one_hot_max_size':4,
                        'eval_metric': 'MultiClass',
                        'class_weights': class_weights,
                        'random_state':42,
                        'task_type':'GPU',
}
    # 파라미터 구성
    cb = CatBoostClassifier(**cbrm_trial_params, verbose = 100)

    # fit the model
    cb.fit(
        train_x.iloc[train_idx], train_y[train_idx],
        eval_set = [(train_x.iloc[valid_idx], train_y[valid_idx])],
        early_stopping_rounds = 50,
        verbose = 100
    )

    # 모델 결과 저장하기
    models.append(cb)
    scores.append(cb.get_best_score()['validation']['MultiClass'])

    # if is_holdout:
    #     break 


0:	learn: 1.3403127	test: 1.3781273	best: 1.3781273 (0)	total: 165ms	remaining: 5m 58s
bestTest = 1.347733759
bestIteration = 12
Shrink model to first 13 iterations.
0:	learn: 1.3555337	test: 1.3605580	best: 1.3605580 (0)	total: 1.73s	remaining: 1h 2m 39s
100:	learn: 0.7092905	test: 0.9914380	best: 0.9914380 (100)	total: 1m 33s	remaining: 32m 11s
200:	learn: 0.6926750	test: 0.9768950	best: 0.9768917 (196)	total: 1m 46s	remaining: 17m 29s
300:	learn: 0.6859321	test: 0.9751215	best: 0.9740160 (292)	total: 2m 3s	remaining: 12m 51s
400:	learn: 0.6138439	test: 0.9782211	best: 0.9721366 (364)	total: 2m 36s	remaining: 11m 34s
bestTest = 0.9721365721
bestIteration = 364
Shrink model to first 365 iterations.
0:	learn: 1.3543104	test: 1.3715021	best: 1.3715021 (0)	total: 1.83s	remaining: 1h 6m 36s
100:	learn: 0.7055291	test: 1.1646663	best: 1.1623075 (62)	total: 1m 25s	remaining: 29m 13s
bestTest = 1.162307456
bestIteration = 62
Shrink model to first 63 iterations.
0:	learn: 1.3572141	test: 1.36

In [ ]:
# cv 결과 확인
print(scores)
print(np.mean(scores))

[0.8564287630260006, 0.3891289356248324, 0.7008520160218688, 0.43765674163579155, 0.38821329444945485, 0.43796552241126563]
0.5350408788615356


In [15]:
# evaluation
def csi_score(CONFUSION_MATRIX, exceptcol = 0):

    # 차원의 수
    n, _ = CONFUSION_MATRIX.shape

    # 계산하여 받을 값
    H = 0
    F = 0
    M = 0
    for i in range(n):
        for j in range(n):
            if i == j == exceptcol:
                continue
            elif i == j:
                H += CONFUSION_MATRIX[i][j]
            elif j != exceptcol:
                F += CONFUSION_MATRIX[i][j]
            elif i != exceptcol:
                M += CONFUSION_MATRIX[i][j]

    return H / (H + F + M)

In [ ]:
# csi_score(confusion_matrix(valid_y, cb_pred), 3)

### 5. 추론

- train, valid를 모두 포함한 데이터를 통해 전체적으로 학습을 다시 진행한 다음 test를 적합하도록 한다.  
- 이름을 꼭 ID에 맞도록 설정해 주어야 한다.

In [ ]:
# predict
pred_list = []

# 각 모델별 예측값 가져오기
for i, model in enumerate(models):
    pred_list.append(model.predict_proba(test_x))

In [ ]:
# 확률값 평균내기
pred_proba = np.mean(pred_list, axis = 0)

In [ ]:
# 예측값 가져오기
pred_class = np.argmax(pred_proba, axis = 1) + 1

In [ ]:
np.unique(pred_class, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([   655,   2723,   2795, 256627], dtype=int64))

### 6. 제출물 저장하기

In [ ]:
# 제출물 불러오기
submission = pd.read_csv('../Data/fog_test.csv')

In [ ]:
# 앞에 필요없는 열 버리기
submission.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
# 대입해서 저장하기
submission['fog_test.class'] = pred_class

In [ ]:
# 저장하기
submission.to_csv('../data/240253.csv', index = False)

### 7. 분석 그 이후

- 생성한 모델에서 어떤 변수가 가장 영향을 주는지 확인할 필요가 있다.

In [ ]:
# # # 분석 결과 feature importance 확인
# for model in models:
#     print("=" * 80)
#     explainer = shap.TreeExplainer(model)
#     shap_values = explainer.shap_values(test_x)
    
#     # class_names = [1, 2, 3, 4]
#     # feature importance plot
#     shap.summary_plot(shap_values, train_x, plot_type="bar",
#                   class_inds="original", class_names=model.classes_, feature_names = train_x.columns)
    
#     print("-" * 80)
#     # visualize 
#     shap.summary_plot(shap_values[0], test_x)
#     shap.summary_plot(shap_values[1], test_x)
#     shap.summary_plot(shap_values[2], test_x)
#     shap.summary_plot(shap_values[3], test_x)

In [16]:
# predict
pred_list2 = []

# 각 모델별 예측값 가져오기
for i, model in enumerate(models):
    pred_list2.append(model.predict_proba(train_x))

In [17]:
# 확률값 평균내기
pred_proba2 = np.mean(pred_list2, axis = 0)

In [18]:
# 예측값 가져오기
pred_class2 = np.argmax(pred_proba2, axis = 1) + 1

In [19]:
np.unique(pred_class2, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([  82681,   90107,  244294, 2739378], dtype=int64))

In [20]:
np.unique(train_y, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([   7687,   11592,   12207, 3124974], dtype=int64))

In [21]:
pd.crosstab(train_y, pred_class2)

col_0,1,2,3,4
class,,,,
1,4899,1599,727,462
2,2359,6133,2129,971
3,1329,2357,7158,1363
4,74094,80018,234280,2736582


In [22]:
# csi score
(2243 + 4888 + 4219)/ (len(train_y) - 3069437)

0.13042528986589752

In [ ]:
# pred
train['pred'] = pred_class2

In [ ]:
pd.crosstab(train[train['ground'] == 'B']['class'], train[train['ground'] == 'B']['pred'])

pred,1,2,3,4
class,,,,
1,742,1340,64,1081
2,184,3666,236,2358
3,85,1381,911,2025
4,2158,11227,6585,597249


In [ ]:
# 1시간 미만만 모아서
fig = px.histogram(train[train['class'] == 2],
                  x = "stn_id",
                  color = 'pred',
                  barmode = "group")

fig.update_layout(title_text="시간별 안개발생 종료 건수 확인",
                    title_x = 0.5,
                    title_xanchor = 'center',
                    title_font_size = 25,
                    title_font_color = 'black',
                    title_font_family = 'NanumSquare',
                    plot_bgcolor='#ffffff')

fig.update_traces(# marker_color = 히스토그램 색, 
                    # marker_line_width = 히스토그램 테두리 두깨,                            
                    # marker_line_color = 히스토그램 테두리 색,
                    marker_opacity = 0.4,
                    )

fig.show()